In [1]:
from pyspark.sql import SparkSession
import nltk
import json
from nltk.corpus import stopwords
import os
from datetime import datetime

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
#Starting spark session
spark_session = SparkSession.builder\
        .master("spark://localhost:7077") \
        .appName("Task-D")\
        .config("spark.cores.max", 4)\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/22 00:14:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def spliting_body(comments_rdd):
    body = comments_rdd \
        .map(lambda line: json.loads(line)['body'].strip()) \
    
    return body

In [4]:
def get_keywords(comment,stop_word_list):
    #nltk.download('stopwords')
    #nltk.download('punkt')
    #nltk.download('averaged_perceptron_tagger')
    # Convert the comment to lowercase and split it into words
    #words = comment.lower().split()
    
    punctuation = set([',', '.', '?', '!', ';', ':', '-', '_', '(', ')', '[', ']', '{', '}','*','#','%'])
    # Remove punctuation and convert comment to lowercase
    for p in punctuation:
        comment = comment.replace(p, '')
    
    words = nltk.word_tokenize(comment.lower())
    # Remove stopwords
    #stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_word_list]
    
    # Extract keywords using NLTK
    keywords = nltk.pos_tag(words)
    keywords = [word for word, tag in keywords if tag.startswith('N')]
    
    return keywords

In [5]:
comments_rdd = spark_session.sparkContext.textFile("hdfs://130.238.28.245:9000/RC_2011-08", use_unicode=True)


In [6]:
comments_rdd.take(5)

['{"link_id":"t3_j4zx3","score_hidden":false,"score":1,"archived":true,"author_flair_text":null,"subreddit":"fffffffuuuuuuuuuuuu","body":"\\"$2, would you take that deal? I\'d take that deal\\"","author":"DorkyDude","distinguished":null,"parent_id":"t3_j4zx3","id":"c298mtc","subreddit_id":"t5_2qqlo","controversiality":0,"gilded":0,"downs":0,"retrieved_on":1427415708,"name":"t1_c298mtc","ups":1,"edited":false,"author_flair_css_class":null,"created_utc":"1312156800"}',
 '{"archived":true,"author_flair_text":null,"score":2,"link_id":"t3_j4kcg","score_hidden":false,"subreddit_id":"t5_2qi6d","controversiality":0,"gilded":0,"parent_id":"t1_c293dct","distinguished":null,"id":"c298mtg","author":"TrptJim","subreddit":"motorcycles","body":"Have you wrecked in them yet?","downs":0,"name":"t1_c298mtg","retrieved_on":1427415708,"created_utc":"1312156800","edited":false,"author_flair_css_class":null,"ups":2}',
 '{"created_utc":"1312156800","edited":false,"author_flair_css_class":"1","ups":1,"downs":

# To extract body field from json

In [7]:
comment = spliting_body(comments_rdd)

In [8]:
comment.take(5)

['"$2, would you take that deal? I\'d take that deal"',
 'Have you wrecked in them yet?',
 'I was thinking 170k max, but if we get another player off the books (Gibson) there would be a small bit more room to maneuver?',
 'a baton is more likely to kill someone than a taser',
 'Cut him a break, he probably just finished watching Training Day.']

In [9]:
# declaring stop_words from NLTK 
stop_words = set(stopwords.words('english'))


# To extract keywords from comments

In [10]:
keywords = comment.flatMap(lambda x: get_keywords(x,stop_words))

In [11]:
keywords.take(5)

['deal', "'d", 'deal', 'get', 'player']

# To find top 25 keywords

In [12]:
top_keywords = keywords.map(lambda x: (x, 1)) \
                      .reduceByKey(lambda x, y: x + y) \
                      .sortBy(lambda x: x[1], ascending=False) \
                      .take(25)

In [15]:
top_keywords

[('people', 1146248),
 ('time', 749068),
 ('way', 537511),
 ('something', 471597),
 ('thing', 387143),
 ('things', 369628),
 ('someone', 345291),
 ('game', 319483),
 ('years', 294378),
 ('lot', 283193),
 ('day', 283188),
 ('anything', 276699),
 ('work', 276135),
 ('point', 248098),
 ('life', 244527),
 ('man', 237846),
 ('money', 232769),
 ('get', 228221),
 ('thanks', 227201),
 ('gt', 218653),
 ('guy', 212467),
 ('year', 211795),
 ('use', 203012),
 ('person', 201057),
 ('everyone', 199371)]

# Kill the spark application

In [16]:
spark_session.stop()